In [ ]:
import torch
torch.cuda.get_device_capability()

(7, 5)

In [ ]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
os._exit(00)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
from torch import cuda, bfloat16
import transformers
import json
from tqdm import tqdm
import pickle
import re
import time


class LlamaModel:
    def __init__(self, hf_token ,model_id):
        self.model_id = model_id
        self.device = self.get_device()
        self.hf_auth = hf_token
        self.bnb_config = self.get_bnb_config()
        self.initalize_model()
        print(f"Model loaded on {self.device}")

    @staticmethod
    def get_device():
        return f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

    @staticmethod
    def get_bnb_config():
        return transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=bfloat16
        )

    def initalize_model(self):
        # begin initializing HF items, need auth token for these

        model_config = transformers.AutoConfig.from_pretrained(
            self.model_id, use_auth_token=self.hf_auth
        )

        model = transformers.AutoModelForCausalLM.from_pretrained(
            self.model_id,
            trust_remote_code=True,
            config=model_config,
            quantization_config=self.bnb_config,
            device_map='auto',
            use_auth_token=self.hf_auth
        )

        tokenizer = transformers.AutoTokenizer.from_pretrained(
            self.model_id, use_auth_token=self.hf_auth
        )

        self.generate_text = transformers.pipeline(
            model=model, tokenizer=tokenizer,
            return_full_text=True,  # langchain expects the full text
            task='text-generation',
            # we pass model parameters here too
            #stopping_criteria=stopping_criteria,  # without this model rambles during chat
            temperature=0.0000001,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
            max_new_tokens=2048,  # mex number of tokens to generate in the output
            repetition_penalty=1.1  # without this output begins repeating
        )

    def call_LLM(self, _prompt):
        res = self.generate_text(_prompt)
        res = res[0]["generated_text"].split('[/INST]')[-1]
        return res

    def llm_response(self,prompt1,text) :

        llm_response1 = self.call_LLM(prompt1.format(text))

        return llm_response1


if __name__ == "__main__" :

    df = pd.DataFrame(['I am Sagar Agrawal',
                             "I am from Bhopal",
                             "Kolkata won the match"],
                            columns = ['text'])

    ## Model initialisations
    hf_token = 'change_this'
    model_id = 'meta-llama/Llama-2-13b-chat-hf'
    llama_model = LlamaModel(hf_token,model_id)

    ## Hierarchy level which we want to predict
    translation_list = []

    ## Defining Prompts
    prompt = """
    <s>[INST] <<SYS>>
    You are a translator. Your job is to translate the provided text to Hindi. Please genereted only translated text in Hindi. Do not give any other details.

    <</SYS>>
    {}
    [/INST]
    """

    start = time.time()
    for text in df.text.tolist() :
        prediction = llama_model.llm_response(prompt,text)
        translation_list.append(prediction)
    df['translated_text' ] = translation_list
    end = time.time()

    print(f"Time taken to predict for {len(df)} titles is : {(end - start)/60} minutes.")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Model loaded on cuda:0
Time taken to predict for 3 titles is : 0.22903844912846882 minutes.


In [ ]:
df

,text,translated_text
0,I am Sagar Agrawal,\n सगर अग्रवाल
1,I am from Bhopal,\n मैं भोपाल से हूँ।
2,Kolkata won the match,\n कोलकाता मैच जीता
